In [278]:
import pandas as pd
import numpy as np 
from sklearn.metrics.pairwise import euclidean_distances
import sklearn
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard
import qgrid


file_path = "C:/Users/user/Documents/R/"
like_in = pd.read_csv(file_path + "ratings_good.csv")
wish_in = pd.read_csv(file_path + "ratings_wish.csv")
score_in = pd.read_csv(file_path + "ratings_score.csv")
lecture_in = pd.read_csv(file_path + "lec_anal.csv")

lecture_list = lecture_in["순번"]
lecture_in = pd.read_csv(file_path + "lec_anal.csv")
lecture_wide = lecture_in.drop(['professor','lecture','리뷰 수'], axis=1)

wish_in["rating"] = 1

def pre(data):
    data_wide = data.pivot_table(index=["id"] , 
                    columns='lecture', 
                    values='rating')
    data_wide = data_wide.fillna(0)
    
    return data_wide

def sim(data_wide):
    #ata_sim = sklearn.metrics.pairwise.cosine_similarity(data_wide.T, dense_output=True)
    #ata_sim = pd.DataFrame(data_sim,index=list(data_wide),columns=list(data_wide))
    #ata_sim = data_sim - np.eye(len(data_sim))
    
    data_sim = sklearn.metrics.pairwise_distances(data_wide.T, metric='cosine')
    data_sim = pd.DataFrame(data_sim,index=list(data_wide),columns=list(data_wide))
    
    data_sim = 1 - data_sim - np.eye(len(data_sim))
    data_sim = pd.DataFrame(data_sim,index = lecture_list, columns = lecture_list)
    data_sim = data_sim.fillna(0)
    
    
    #데이터의 수가 희소하므로 표준화는 배제한다. 
    #표준화가 필요하다면 0을 제외하고 진행할 것. 4/9 와 9 가 있을 때, 4/9의 9가 더 작아질 위험이 있음.

    # euclidean_distance & 정규화
    """data_sim = euclidean_distances(data_wide.T,data_wide.T)
    data_sim = data_sim / data_sim.max(axis=0).max()
    data_sim = (data_sim - data_sim.min(axis=0).min()) / (data_sim.max(axis=0).max() - data_sim.min(axis=0).min())
    data_sim = 1 - data_sim - np.eye(len(data_sim))"""

    return data_sim


def sim_bin(data_wide):
    data_sim = 1 - pdist(data_wide.T,'jaccard')
    data_sim = squareform(data_sim)
    data_sim = pd.DataFrame(data_sim, index=list(data_wide), columns=list(data_wide))
    
    data_sim = pd.DataFrame(data_sim,index = lecture_list, columns = lecture_list)
    data_sim = data_sim.fillna(0)    
    
    return data_sim
    
def sim_lecture(data_wide):  
    data_wide = data_wide.set_index("순번")
    #print(lecture_wide)
    data_sim = 1- pdist(data_wide,'jaccard')
    data_sim = squareform(data_sim)
    data_sim = pd.DataFrame(data_sim, index=list(lecture_list), columns=list(lecture_list))

    return data_sim 

def recommend(input_rating,sim):
    series = np.dot(sim,input_rating)
    result = pd.DataFrame(series,index=list(sim))
    result = result.sort_values(by=0, axis=0,ascending=False)
    return result

In [280]:
like_wide = pre(like_in)
like_sim = sim(like_wide)

score_wide = sim(score_in)
score_sim = sim(score_wide)


wish_wide = pre(wish_in)
wish_sim = sim_bin(wish_wide)

lecture_sim = sim_lecture(lecture_wide)

like_sim.to_csv("like_sim.csv")
wish_sim.to_csv("wish_sim.csv")
lecture_sim.to_csv("lecture_sim.csv")
score_sim.to_csv("score_sim.csv")

In [254]:

#total_sim = like_sim * alpha + score_sim * beta + wish_sim * theta + keyword_sim * delta  

#test
input_sample = like_wide.loc[8]

input_sample = pd.DataFrame(input_sample, index=list(lecture_list)).fillna(0)


print(recommend(input_sample, wish_sim))



         0
10014  0.8
10089  0.4
10011  0.4
10001  0.0
10147  0.0
...    ...
10076  0.0
10077  0.0
10078  0.0
10079  0.0
10212  0.0

[212 rows x 1 columns]
